<a href="https://colab.research.google.com/github/aswit3/Start_Your_NLP_Career/blob/master/training_imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install fasttext

     |████████████████████████████████| 61kB 2.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/9f/f0/04/caa82c912aee89ce76358ff954f3f0729b7577c8ff23a292e3
Successfully built fasttext


In [2]:
import fasttext
import numpy as np
import pandas as pd
import tensorflow as tf
import keras.layers as layers
from keras.models import Model
from keras.datasets import imdb
from gensim.models import Word2Vec
from gensim.models import FastText
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input,Embedding,Dense,Flatten
from sklearn.metrics import accuracy_score,classification_report
#from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [0]:
def load_data(vocab_size,max_len):
    """
        Loads the keras imdb dataset

        Args:
            vocab_size = {int} the size of the vocabulary
            max_len = {int} the maximum length of input considered for padding

        Returns:
            X_train = tokenized train data
            X_test = tokenized test data

    """
    INDEX_FROM = 3

    # save np.load
    np_load_old = np.load

    # modify the default parameters of np.load
    np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

    (X_train,y_train),(X_test,y_test) = imdb.load_data(num_words = vocab_size,index_from = INDEX_FROM)

    # restore np.load for future normal usage
    np.load = np_load_old

    print(len(X_train), len(X_test), len(y_train), len(y_test), "#####################################")

    return X_train,X_test,y_train,y_test


In [0]:
def prepare_data_for_word_vectors_imdb(X_train):
    """
        Prepares the input

        Args:
            X_train = tokenized train data

        Returns:
            sentences = {list} sentences containing words as tokens
            word_index = {dict} word and its indexes in whole of imdb corpus

    """
    INDEX_FROM = 3
    word_to_index = imdb.get_word_index()
    word_to_index = {k:(v+INDEX_FROM) for k,v in word_to_index.items()}

    word_to_index["<START>"] =1
    word_to_index["<UNK>"]=2

    index_to_word = {v:k for k,v in word_to_index.items()}

    sentences = []
    for i in range(len(X_train)):
        temp = [index_to_word[ids] for ids in X_train[i]]
        sentences.append(temp)
    """
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(sentences)
    word_indexes = tokenizer.word_index
    """

    #print(sentences[:10],word_to_index,"sentences[:10],word_to_index[:10]*********************************************")
    return sentences,word_to_index

In [0]:
def building_word_vector_model(option,sentences,embed_dim,workers,window,y_train):
    """
        Builds the word vector

        Args:
            type = {bool} 0 for Word2vec. 1 for gensim Fastext. 2 for Fasttext 2018.
            sentences = {list} list of tokenized words
            embed_dim = {int} embedding dimension of the word vectors
            workers = {int} no. of worker threads to train the model (faster training with multicore machines)
            window = {int} max distance between current and predicted word
            y_train = y_train

        Returns:
            model = Word2vec/Gensim fastText/ Fastext_2018 model trained on the training corpus


    """
    if option == 0:
        print("Training a word2vec model")
        model = Word2Vec(sentences=sentences, size = embed_dim, window = window) # workers = workers,
        print("Training complete")

    elif option == 1:
        print("Training a Gensim FastText model")
        model = FastText(sentences=sentences, size = embed_dim, window = window) # workers = workers, 
        print("Training complete")

    elif option == 2:
        print("Training a Fasttext model from Facebook Research")
        y_train = ["__label__positive" if i==1 else "__label__negative" for i in y_train]

        with open("imdb_train.txt","w") as text_file:
            for i in range(len(sentences)):
                print(sentences[i],y_train[i],file = text_file)

        model = fasttext.train_unsupervised("imdb_train.txt", model = "skipgram", lr=0.05, dim=embed_dim, ws=5, epoch=5) 
        print("Training complete")

    return model

In [6]:
# specify “option” as  0 – Word2vec, 1 – Gensim FastText, 2- Fasttext

option = 1

embed_dim = 300
split_ratio= 0.33
max_len= 200
vocab_size= 1000
trainable_param= False
workers = 3,
window = 1


x_train,x_test,y_train,y_test = load_data(vocab_size,max_len)
sentences,word_ix = prepare_data_for_word_vectors_imdb(x_train)
model_wv = building_word_vector_model(option,sentences,embed_dim,workers,window,y_train)

25000 25000 25000 25000 #####################################
Training a Gensim FastText model
Training complete


In [0]:
def padding_input(X_train,X_test,maxlen):
    """
        Pads the input upto considered max length

        Args:
            X_train = tokenized train data
            X_test = tokenized test data

        Returns:
            X_train_pad = padded tokenized train data
            X_test_pad = padded tokenized test data

    """
    print(X_train.shape, X_test.shape, "before padding")

    X_train_pad = pad_sequences(X_train,maxlen=maxlen,padding="post")

    X_test_pad = pad_sequences(X_test,maxlen=maxlen,padding="post")

    print(X_train_pad.shape, X_test_pad.shape, "after padding")

    return X_train_pad,X_test_pad

In [8]:
x_train_pad,x_test_pad = padding_input(x_train,x_test,max_len)

(25000,) (25000,) before padding
(25000, 200) (25000, 200) after padding


In [0]:
def classification_model(embed_dim,X_train_pad,X_test_pad,y_train,y_test,vocab_size,word_index,w2vmodel,trainable_param):
    """
        Builds the classification model for sentiment analysis

        Args:
            embded_dim = {int} dimension of the word vectors
            X_train_pad = padded tokenized train data
            X_test_pad = padded tokenized test data
            vocab_size = {int} size of the vocabulary
            word_index =  {dict} word and its indexes in whole of imdb corpus
            w2vmodel = Word2Vec model
            trainable_param = {bool} whether to train the word embeddings in the Embedding layer
    """

    embedding_matrix = np.zeros((vocab_size,embed_dim))
    print(embedding_matrix.shape, "embedding_matrix")
    for word, i in word_index.items():
        try:
            embedding_vector = w2vmodel[word]
            
        except:
            pass
        try:
            if embedding_vector is not None:
                embedding_matrix[i]=embedding_vector
        except:
            pass
    #print(embedding_vector, word, "embedding_vector, word")
    print(embedding_matrix.shape ,"embedding_matrix")       
    embed_layer = Embedding(vocab_size,embed_dim,weights =[embedding_matrix],trainable=trainable_param)

    input_seq = Input(shape=(X_train_pad.shape[1],))
    embed_seq = embed_layer(input_seq)
    x = Dense(256,activation ="relu")(embed_seq)
    x = Flatten()(x)
    preds = Dense(1,activation="sigmoid")(x)

    model = Model(input_seq,preds)


    model.compile(loss=loss,optimizer=optimizer,metrics= metrics)

    model.fit(X_train_pad,y_train,epochs=epochs,batch_size=batch_size,validation_data=(X_test_pad,y_test))
    predictions = model.predict(X_test_pad)
    predictions = [0 if i<0.5 else 1 for i in predictions]
    print("Accuracy: ",accuracy_score(y_test,predictions))
    print("Classification Report: ",classification_report(y_test,predictions))

    return model


In [14]:
epochs = 1
batch_size = 1024
loss = "binary_crossentropy"
optimizer = "adam"
metrics = ["accuracy"]
      
model = classification_model(embed_dim,x_train_pad,x_test_pad,y_train, y_test,vocab_size,word_ix,model_wv,trainable_param)
print(model.summary())

(1000, 300) embedding_matrix


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
W0717 14:56:39.917273 140633074726784 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



(1000, 300) embedding_matrix


W0717 14:56:39.969289 140633074726784 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 96s 4ms/step - loss: 0.7964 - acc: 0.5249 - val_loss: 0.6798 - val_acc: 0.5708
Accuracy:  0.57084
Classification Report:                precision    recall  f1-score   support

           0       0.55      0.84      0.66     12500
           1       0.66      0.30      0.41     12500

    accuracy                           0.57     25000
   macro avg       0.60      0.57      0.54     25000
weighted avg       0.60      0.57      0.54     25000

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 200)               0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 200, 300)          300000    
_________________________________________________________________
dense_5 (Dense)              (None, 200, 256)   